In [7]:
import re
import requests as r
import time 
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"

In [2]:
def meanRetAn(data):             
    Result = 1
    
    for i in range(len(data.index)):
        Result *= (1+data.iloc[i,:])
        
    Result = Result**(1/float(len(data.index)/52))-1
     
    return(Result)

In [18]:
#getting etf tickers from etf in etf info
#etf_info = pd.read_csv("ETFs_info.csv")
yahoo_cat_df = pd.read_csv("yahoo_cat_df.csv",index_col="Ticker")
week_df_900 = pd.read_csv("WeeklyReturns.csv", index_col="Date")
etf_funda_df = pd.read_csv("etf_fundamental.csv", index_col="Unnamed: 0") # result from the fetcing code below
etf_funda_df = pd.merge(yahoo_cat_df,etf_funda_df, right_index=True,left_index=True, how="right")
etf_funda_df["pe"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pcf"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["ps"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pb"].loc[etf_funda_df['Category'] == "Trading--Inverse Equity"] = -1
etf_funda_df["pe"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["pcf"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["ps"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df["pb"].loc[etf_funda_df['Category'] == "Trading--Leveraged Equity"] = -5
etf_funda_df.drop(columns="Category", inplace=True)
print(len(etf_funda_df))
etf_funda_df = etf_funda_df.dropna()


1577


In [19]:
etf_funda_df

,pe,pb,pcf,ps
SPY,27.93,4.00,16.31,2.74
IVV,27.16,3.81,16.08,2.62
VTI,27.49,3.81,15.92,2.58
VOO,27.94,4.00,16.32,2.75
QQQ,34.42,7.93,21.00,4.64
...,...,...,...,...
DMAY,27.94,4.00,16.31,2.74
NJUL,34.65,7.86,21.31,4.77
BMAY,27.17,3.81,16.08,2.62
FLV,21.89,2.45,15.00,2.43


In [4]:
# dff = pd.DataFrame(mu_ga).rename(columns={"2016-08-05":"AnnRet"})
# dff["Annual Std"] = week_df_900.std(axis=0) * np.sqrt(52)


In [20]:
#etf_merged.dropna(inplace=True)
etf_merged = etf_funda_df.copy()

In [6]:
# # # fetching fundamentals from yahoo finance on each etf
# ticker = list(yahoo_cat_df.index)
# test = {}
# for i in tqdm(ticker):
#     test[i] = {}
#     lnk = f"https://finance.yahoo.com/quote/{i}/holdings?p={i}"
#     con = r.get(lnk)
#     soup = BeautifulSoup(con.text, "lxml")
#     try: 


#         test[i]["pe"] = soup.find('span', {"data-reactid":"131"}).get_text()
#         test[i]["pb"] = soup.find('span', {"data-reactid":"136"}).get_text()
#         test[i]["pcf"] = soup.find('span', {"data-reactid":"146"}).get_text()
#         test[i]["ps"] = soup.find('span', {"data-reactid":"141"}).get_text()
#     except:
#         test[i]["pe"] = np.nan
#         test[i]["pb"] = np.nan
#         test[i]["pcf"] = np.nan
#         test[i]["ps"] = np.nan
# a = pd.DataFrame(test).T
# a.to_csv("etf_fundamental.csv")

In [35]:
# Standardizing the features
cluster_stand = StandardScaler().fit_transform(etf_merged)
cluster_stand = pd.DataFrame(cluster_stand)

In [36]:
cluster_stand

,0,1,2,3
0,0.796028,0.483677,0.783723,0.484963
1,0.728745,0.418524,0.753354,0.431601
2,0.757581,0.418524,0.732228,0.413814
3,0.796902,0.483677,0.785044,0.489410
4,1.363129,1.831321,1.402990,1.329866
...,...,...,...,...
1555,0.796902,0.483677,0.783723,0.484963
1556,1.383227,1.807317,1.443922,1.387675
1557,0.729619,0.418524,0.753354,0.431601
1558,0.268249,-0.047836,0.610751,0.347111


In [37]:
#Reducing dementionality of the data features to be explanined by 2 components for visualization and clustering 
X_embedded = TSNE(n_components=2, random_state=400).fit_transform(cluster_stand)
cluster_stand["dim1"] = X_embedded[:,0]
cluster_stand["dim2"] = X_embedded[:,1]
# cluster_stand = StandardScaler().fit_transform(etf_merged)
# cluster_stand = pd.DataFrame(etf_merged)


In [38]:
cluster_stand

,0,1,2,3,dim1,dim2
0,0.796028,0.483677,0.783723,0.484963,18.668848,-31.660778
1,0.728745,0.418524,0.753354,0.431601,19.706816,-24.147781
2,0.757581,0.418524,0.732228,0.413814,20.383694,-22.401855
3,0.796902,0.483677,0.785044,0.489410,18.700769,-32.715122
4,1.363129,1.831321,1.402990,1.329866,0.270833,-46.132282
...,...,...,...,...,...,...
1555,0.796902,0.483677,0.783723,0.484963,18.325798,-32.341679
1556,1.383227,1.807317,1.443922,1.387675,0.479006,-46.607925
1557,0.729619,0.418524,0.753354,0.431601,20.441612,-24.165724
1558,0.268249,-0.047836,0.610751,0.347111,7.149090,-7.621745


In [39]:
#finding the "optimal" cluster 
iini = []
clus = []
for n_clusters in range(2,14):
        clusterer = KMeans(n_clusters=n_clusters, random_state=10,n_init=20, algorithm="elkan",init='k-means++')
        cluster_labels = clusterer.fit_predict(cluster_stand)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(cluster_stand, cluster_labels)
        print("For n_clusters =", n_clusters,
            "The average silhouette_score is :", silhouette_avg)
        clus.append(n_clusters)
        iini.append(clusterer.inertia_)
fig = px.line(y=iini,x=clus)
fig.show()
        

For n_clusters = 2 The average silhouette_score is : 0.4437416238983516
For n_clusters = 3 The average silhouette_score is : 0.4768930952236609
For n_clusters = 4 The average silhouette_score is : 0.4663594218771804
For n_clusters = 5 The average silhouette_score is : 0.5120947574758284
For n_clusters = 6 The average silhouette_score is : 0.532826100163838
For n_clusters = 7 The average silhouette_score is : 0.5128123772992926
For n_clusters = 8 The average silhouette_score is : 0.4992627149746311
For n_clusters = 9 The average silhouette_score is : 0.4881527666984056
For n_clusters = 10 The average silhouette_score is : 0.49540928769275716
For n_clusters = 11 The average silhouette_score is : 0.48372961706526574
For n_clusters = 12 The average silhouette_score is : 0.4777085157850483
For n_clusters = 13 The average silhouette_score is : 0.4843768587342331


In [43]:

km = KMeans(
    n_clusters=6, init='k-means++',
    n_init=20, max_iter=1000, 
    tol=1e-04, random_state=10, algorithm="elkan"
)
clustered = km.fit_predict(cluster_stand)
cluster_stand["cluster"] = clustered
cluster_stand["dim1"] = X_embedded[:,0]
cluster_stand["dim2"] = X_embedded[:,1]
fig = px.scatter(cluster_stand,x="dim1", y="dim2", color="cluster")
fig.show()

[5 5 5 ... 5 0 2]


In [44]:
week_df_900 = pd.merge(week_df_900.T,yahoo_cat_df, right_index=True, left_index= True, how="inner")
week_df_900 = week_df_900.iloc[:,:-1].T

In [52]:
week_df_900

,MDY,IUSG,VFH,FVD,SPMD,PDP,VONG,SPXL,MLPA,IGE,...,IYM,ECH,CFA,XLV,DVYE,PSCI,VOT,DBEZ,VPU,SPHQ
2015-01-14,-0.003038,-0.002729,-0.020341,0.002930,-0.006594,-0.007127,-0.002485,-0.024268,-0.025823,-0.017808,...,-0.009989,-0.021661,-0.008938,0.002441,0.008660,0.007786,-0.001400,0.019445,0.011201,-0.006082
2015-01-21,0.006788,0.009904,-0.002307,0.015860,0.003319,0.015099,0.010795,0.032345,0.045063,0.041004,...,0.026863,0.016605,0.000422,0.009882,0.010971,-0.010907,0.009114,0.051450,0.026392,0.010489
2015-01-28,0.006091,-0.004645,-0.005676,-0.010682,0.017030,0.004389,-0.005032,-0.045574,-0.013951,-0.024920,...,-0.027266,0.009852,0.020707,-0.010353,0.004246,0.009419,-0.000893,0.014101,-0.008634,-0.012543
2015-02-04,0.015801,0.018797,0.020719,0.009136,0.005662,0.003399,0.016617,0.057697,0.016078,0.052212,...,0.035227,0.013350,0.002760,-0.007595,0.001175,0.021848,0.010332,0.018038,-0.012779,0.012703
2015-02-11,0.009071,0.017814,0.015534,-0.004115,0.016171,0.015727,0.018173,0.041970,-0.009806,0.007835,...,0.023295,-0.000760,0.012938,0.013574,-0.016190,0.015590,0.014454,0.004799,-0.042477,0.010381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,-0.004703,0.015283,0.001419,-0.005421,-0.004651,0.004858,0.018816,0.033323,-0.028641,-0.013158,...,-0.001536,-0.000714,0.004907,0.028126,0.012216,-0.018419,0.014011,0.001649,-0.013200,0.014303
2020-12-09,0.021902,-0.004748,0.007371,0.008032,0.022586,0.000242,-0.005211,0.000740,0.048184,0.041290,...,0.015742,0.023937,0.011472,0.007355,0.024418,0.049374,0.008488,0.006035,-0.017332,0.001459
2020-12-16,0.018448,0.018175,0.003940,-0.001138,0.017517,0.037699,0.022253,0.025868,-0.000343,-0.004492,...,-0.005161,0.005506,0.005727,-0.005342,0.033020,-0.002917,0.027744,0.007392,0.000951,0.008983
2020-12-23,0.014334,-0.000115,0.010978,-0.010256,0.014445,0.024220,0.007295,-0.009300,-0.034317,-0.021757,...,0.002516,0.030345,0.006899,-0.000910,-0.016052,0.016366,0.018330,-0.000013,-0.016982,-0.003263


In [53]:
etf_funda_df

,pe,pb,pcf,ps
SPY,27.93,4.00,16.31,2.74
IVV,27.16,3.81,16.08,2.62
VTI,27.49,3.81,15.92,2.58
VOO,27.94,4.00,16.32,2.75
QQQ,34.42,7.93,21.00,4.64
...,...,...,...,...
DMAY,27.94,4.00,16.31,2.74
NJUL,34.65,7.86,21.31,4.77
BMAY,27.17,3.81,16.08,2.62
FLV,21.89,2.45,15.00,2.43


In [45]:
etf_merged["cluster"] = clustered
list1_as_set = set(week_df_900.columns)
intersection = list1_as_set.intersection(list(etf_funda_df.index))
intersection_as_list = list(intersection)
fund_min = etf_merged.loc[intersection_as_list,:]
week_df_900 = week_df_900[intersection_as_list]


In [54]:
fund_min

,pe,pb,pcf,ps,cluster
MDY,25.01,2.55,12.21,1.71,0
IUSG,34.77,8.72,22.05,4.72,5
VFH,17.37,1.44,0.00,2.34,1
FVD,21.00,2.48,11.82,1.87,0
SPMD,24.60,2.41,11.43,1.58,0
...,...,...,...,...,...
PSCI,29.17,2.61,10.79,1.12,0
VOT,38.80,6.50,24.84,4.72,5
DBEZ,21.40,1.67,9.44,1.23,2
VPU,21.67,1.95,9.70,2.56,2


In [46]:
dfff = pd.DataFrame(index=week_df_900.index)
dff_dict = {}
for i in fund_min["cluster"].unique():
    tickers = list(fund_min[fund_min["cluster"]==i].index)
    n_assets = len(tickers)
    portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
    port_weekly_return = week_df_900[tickers].mul(portfolio_weights_ew,axis=1).sum(axis=1)
    dfff[i] = port_weekly_return
    
cumsum = dfff.cumsum(axis=0)

In [47]:
cluster_sort = pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)["index"]


In [48]:
pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)

,index,0
5,4,0.422749
3,3,0.421738
2,1,0.218842
4,2,0.182214
1,5,0.174717
0,0,0.169699


In [49]:
cumsum["SPY"]= week_df_900["SPY"].cumsum(axis=0)


In [50]:
cumsum

,0,5,1,3,2,4,SPY
2015-01-14,-0.001175,-0.001561,-0.005804,-0.009099,0.000367,0.003087,-0.007167
2015-01-21,0.012663,0.008769,0.010966,0.018597,0.020533,-0.027880,0.003885
2015-01-28,0.006464,0.005561,0.002100,-0.001149,0.018090,-0.005277,-0.010592
2015-02-04,0.022032,0.019333,0.025790,0.039798,0.034487,-0.044093,0.008994
2015-02-11,0.028925,0.029715,0.026114,0.061952,0.031990,-0.071308,0.023059
...,...,...,...,...,...,...,...
2020-12-02,0.616427,0.920624,0.329503,1.590497,0.508222,-2.458567,0.784790
2020-12-09,0.630364,0.928248,0.352845,1.616113,0.523325,-2.479955,0.784953
2020-12-16,0.638801,0.949804,0.367155,1.640234,0.534338,-2.519209,0.794003
2020-12-23,0.643550,0.965764,0.365062,1.646422,0.533096,-2.518669,0.791212


In [51]:

plot = px.line(cumsum, x = cumsum.index, y = cumsum.columns) 
plot.show()

In [19]:
for i in cluster_sort:
    print(i)
    if "SPY" in etf_merged[ etf_merged["cluster"]==i].index: 
        print("SPY in:",i)
    print("Number of Assets:",len(etf_merged[ etf_merged["cluster"]==i]))
    print("Price/Sales",etf_merged[ etf_merged["cluster"]==i]["ps"].mean()) 
    print("Price/Book",etf_merged[ etf_merged["cluster"]==i]["pb"].mean()) 
    print("Price/Casflow",etf_merged[ etf_merged["cluster"]==i]["pcf"].mean()) 
    print("Price/Earnings",etf_merged[ etf_merged["cluster"]==i]["pe"].mean()) 
    avg = etf_merged[ etf_merged["cluster"]==i]["ps"].mean()+etf_merged[ etf_merged["cluster"]==i]["pb"].mean()+etf_merged[ etf_merged["cluster"]==i]["pcf"].mean()+etf_merged[          etf_merged["cluster"]==i]["pe"].mean()
    print(avg/4)
    print(20*"-")

4
Number of Assets: 75
Price/Sales -5.0
Price/Book -5.0
Price/Casflow -5.0
Price/Earnings -5.0
-5.0
--------------------
0
Number of Assets: 65
Price/Sales -1.0
Price/Book -1.0
Price/Casflow -1.0
Price/Earnings -1.0
-1.0
--------------------
5
Number of Assets: 161
Price/Sales 1.9332919254658385
Price/Book 1.8878260869565218
Price/Casflow 2.8816770186335408
Price/Earnings 4.156832298136646
2.7149068322981367
--------------------
3
SPY in: 3
Number of Assets: 432
Price/Sales 3.550694444444445
Price/Book 5.695324074074074
Price/Casflow 18.94199074074074
Price/Earnings 29.77212962962963
14.490034722222223
--------------------
2
Number of Assets: 389
Price/Sales 1.0327506426735218
Price/Book 1.567146529562982
Price/Casflow 7.349588688946015
Price/Earnings 16.474884318766065
6.6060925449871455
--------------------
1
Number of Assets: 438
Price/Sales 1.7493150684931509
Price/Book 2.5243835616438353
Price/Casflow 11.68566210045662
Price/Earnings 22.511050228310502
9.617602739726028
----------

In [20]:
test = pd.DataFrame(dfff.std(axis=0)*np.sqrt(52)).reset_index().sort_values(by=[0], ascending=False)
annn = pd.DataFrame(meanRetAn(dfff)).reset_index().rename(columns={"2016-08-05":"AnnRet"})
dingo = pd.merge(test,annn,on="index",how="inner")

In [21]:
yahoo_df = pd.read_csv("yahoo_cat_df.csv", index_col="Ticker")

In [22]:
com_df = pd.merge(etf_merged,yahoo_cat_df,right_index=True,left_index=True,how="inner")


In [23]:
for i in cluster_sort:
    print(i)
    print(len(com_df[com_df["cluster"]==i]), sum(com_df[com_df["cluster"]==i]["Category"].value_counts()))
    print(com_df[com_df["cluster"]==i]["Category"].value_counts())
    print(30*"-")

4
75 75
Trading--Leveraged Equity    75
Name: Category, dtype: int64
------------------------------
0
65 65
Trading--Inverse Equity    65
Name: Category, dtype: int64
------------------------------
5
161 134
Financial                     30
Equity Energy                 21
Energy Limited Partnership    14
Health                        11
Miscellaneous Sector           8
Miscellaneous Region           6
Technology                     6
Large Growth                   5
World Stock                    4
Consumer Cyclical              4
Large Blend                    3
Europe Stock                   3
Large Value                    3
Option Writing                 2
Bear Market                    2
Natural Resources              2
Small Growth                   2
Equity Precious Metals         2
Foreign Large Value            1
Pacific/Asia ex-Japan Stk      1
Diversified Emerging Mkts      1
China Region                   1
Small Blend                    1
Nontraditional Bond            1


In [24]:
funda_cluster = {}
for i,x in zip(cluster_sort,["Market Leveraged ETF","Market Inverse ETF","Low Fundamental ETF’s (Energy, Health,Finance)","High Fundamental Broad Market","Low Fundamental Broad Market","Mid Fundamental Broad Market"]):
    funda_cluster[i] = x

In [25]:
funda_cluster

{4: 'Market Leveraged ETF',
 0: 'Market Inverse ETF',
 5: 'Low Fundamental ETF’s (Energy, Health,Finance)',
 3: 'High Fundamental Broad Market',
 2: 'Low Fundamentals Broad Market',
 1: 'Mid Fundamentals Broad Market'}

In [29]:
com_df["Fundamental Cluster"] = com_df["cluster"].apply(lambda x: funda_cluster[x])

In [33]:
com_df.to_csv("Fundamental_Clustering.csv", index_label="Ticker")

In [27]:
fundamental_df_cluster = com_df.copy()

In [35]:
pd.read_csv("Fundamental_Clustering.csv", index_col="Ticker")

,pe,pb,pcf,ps,cluster,Category,Fundamental Cluster
Ticker,,,,,,,
SPY,27.93,4.00,16.31,2.74,3,Large Blend,High Fundamental Broad Market
IVV,27.16,3.81,16.08,2.62,3,Large Blend,High Fundamental Broad Market
VTI,27.49,3.81,15.92,2.58,3,Large Blend,High Fundamental Broad Market
VOO,27.94,4.00,16.32,2.75,3,Large Blend,High Fundamental Broad Market
QQQ,34.42,7.93,21.00,4.64,3,Large Growth,High Fundamental Broad Market
...,...,...,...,...,...,...,...
DMAY,27.94,4.00,16.31,2.74,3,NaN,High Fundamental Broad Market
NJUL,34.65,7.86,21.31,4.77,3,Large Blend,High Fundamental Broad Market
BMAY,27.17,3.81,16.08,2.62,3,Large Blend,High Fundamental Broad Market
